In [4]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,
                          BatchNormalization, Input, Conv2D, GlobalAveragePooling2D,MaxPooling2D)
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
import keras
from keras.models import Model
import matplotlib.pyplot as plt 

from keras.preprocessing import image
from keras.layers import merge, Input
from keras.utils import np_utils
import os
import time

import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
SEED=2

train=pd.read_csv("inputs/train.csv")
submition=pd.read_csv("inputs/test.csv")

x = train['id_code']
y = train['diagnosis']

x,y=shuffle(x,y)

In [5]:
df_X,X_test,df_y,y_test=train_test_split(x, y, test_size=0.15)

X_train,X_valid,y_train,y_valid=train_test_split(df_X, df_y, test_size=0.15)

In [6]:
def load_ben_color(path, sigmaX=20):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image

In [7]:
def crop_image1(img,tol=7):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [10]:
import matplotlib.pyplot as plt 
import cv2
test_images=[]
IMG_SIZE=300
%time
fig = plt.figure(figsize=(25, 16))

for idx, row in enumerate(X_test):
    
    path=f"inputs/train_images/{row}.png"
    
    image =load_ben_color(path)
    test_images.append(image)

Wall time: 0 ns


<Figure size 1800x1152 with 0 Axes>

In [11]:
train_images=[]
%time
fig = plt.figure(figsize=(25, 16))

for idx, row in enumerate(X_test):
    
    path=f"inputs/train_images/{row}.png"
    image =load_ben_color(path)
    train_images.append(image)

Wall time: 0 ns


<Figure size 1800x1152 with 0 Axes>

In [12]:
valid_images=[]

%time
fig = plt.figure(figsize=(25, 16))

for idx, row in enumerate(X_test):
    
    path=f"inputs/train_images/{row}.png"
    image =load_ben_color(path)
    valid_images.append(image)

Wall time: 0 ns


<Figure size 1800x1152 with 0 Axes>

In [15]:
#Importing the ResNet50 model
from keras.applications.resnet50 import ResNet50, preprocess_input

#Loading the ResNet50 model with pre-trained ImageNet weights
model = ResNet50(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

In [16]:
#Reshaping the testing data
test=np.array(test_images)

#Preprocessing the data, so that it can be fed to the pre-trained ResNet50 model.
resnet_test_input = preprocess_input(test)

#Creating bottleneck features for the testing data
test_features = model.predict(resnet_test_input)

#Saving the bottleneck features
np.savez('bottleneck/resnet_features_test', features=test_features)

In [17]:
#Reshaping the testing data
train=np.array(train_images)

#Preprocessing the data, so that it can be fed to the pre-trained ResNet50 model.
resnet_train_input = preprocess_input(train)

#Creating bottleneck features for the testing data
train_features = model.predict(resnet_train_input)

#Saving the bottleneck features
np.savez('bottleneck/resnet_features_train', features=train_features)

In [18]:
#Reshaping the testing data
valid=np.array(valid_images)

#Preprocessing the data, so that it can be fed to the pre-trained ResNet50 model.
resnet_valid_input = preprocess_input(valid)

#Creating bottleneck features for the testing data
valid_features = model.predict(resnet_valid_input)

#Saving the bottleneck features
np.savez('bottleneck/resnet_features_valid', features=valid_features)